In [6]:
import os
from pathlib import Path
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import json

load_dotenv()

PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT")).resolve()  # type: ignore
DATA_ROOT = Path(os.getenv("DATA_ROOT")).resolve()  # type: ignore


def ema(values, alpha=0.8):
    """Exponential Moving Average smoothing"""
    smoothed = []
    m = None
    for v in values:
        m = v if m is None else alpha * m + (1 - alpha) * v
        smoothed.append(m)
    return smoothed

In [7]:
win_rate_root = DATA_ROOT / "experiment3" / "validation"
steps: list[list] = []
win_rates: list[list] = []
names = []

name_dict = {
    "cyclic": "Baseline (All edges)",
    "acyclic_reason_kahn": "FASO Reason-Kahn",
    "acyclic_no_reason_kahn": "FASO-Kahn",
    "acyclic_reason_deterministic": "FASO-Deterministic",
    "acyclic_no_reason_deterministic": "FASO Reason-Deterministic",
}

for sub in sorted(win_rate_root.iterdir()):
    if not sub.is_dir():
        with open(sub, "r") as f:
            loaded = json.load(f)
            assert isinstance(loaded, list)
        names.append(name_dict[sub.name[0:-13][0:-1]])
        step = []
        win_rate = []
        for item in loaded:
            assert isinstance(item, dict)
            step.append(item["step"])
            win_rate.append(item["win_rate"])
        sorted_pairs = sorted(zip(step, win_rate))
        step, win_rate = zip(*sorted_pairs)
        steps.append(step)
        win_rates.append(win_rate)

In [8]:
def plot_win_rate(steps, win_rates, names, save_path, for_paper=False, use_ema=False):
    if for_paper:
        plt.rcParams.update({
            "font.family": "Times New Roman",
            "mathtext.fontset": "stix",
            "figure.dpi": 300,
            "axes.titlesize": 14,
            "axes.labelsize": 12,
            "xtick.labelsize": 10,
            "ytick.labelsize": 10,
            "legend.fontsize": 11,
            "lines.linewidth": 2,
            "lines.markersize": 6,
        })
        figsize=(6,4)
        alpha_raw = 0.15
        lw = 1.0
        add_grid = False
        add_title = False
    else:
        figsize = (10, 6)
        lw = 1.8
        add_grid = True

    plt.figure(figsize=figsize)
    for i in range(len(names)):
        plt.plot(steps[i], win_rates[i], color=f"C{i}", alpha=0.15)
        if use_ema:
            win_rates[i] = ema(win_rates[i])
        plt.plot(steps[i], win_rates[i], label=names[i], color=f"C{i}", alpha=1.0)

    plt.xlabel("Step")
    plt.ylabel("Win Rate Against SFT Model")
    plt.legend(frameon=False)
    if add_grid:
        plt.grid()
    plt.tight_layout()

    # 논문용은 pdf, 아니면 png
    ext = "pdf"
    plt.savefig(save_path.with_suffix(f".{ext}"), bbox_inches='tight')
    plt.close()

In [9]:
plot_win_rate(
    steps=steps,
    win_rates=win_rates,
    names=names,
    save_path=PROJECT_ROOT
    / "graphs"
    / "experiment3"
    / "comparisons"
    / "validation_win_rate",
    for_paper=True,
    use_ema=True,
)

In [10]:
avg = [sum(win_rate) / len(win_rate) for win_rate in win_rates]
print(avg)
print(names)
grow = [(a - avg[len(avg) - 1]) * 100 for a in avg]
print(grow)
print(sum(grow) / (len(grow) - 1))
# [0.8698250506029237, 1.1584017709978078, 1.7108501369614726, 1.492915553190688, 0.0]
# [1.2132944787093147, 0.8530649057726825, 1.2107779796903695, 0.5674226170141572, 0.0]

[0.913042479866246, 0.9159282470701948, 0.9214527307298315, 0.9192733848921236, 0.9043442293602167]
['FASO Reason-Deterministic', 'FASO-Kahn', 'FASO-Deterministic', 'FASO Reason-Kahn', 'Baseline (All edges)']
[0.8698250506029237, 1.1584017709978078, 1.7108501369614726, 1.492915553190688, 0.0]
1.3079981279382231
